In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import yfinance as yf

Using TensorFlow backend.


In [5]:
#tickers = ['AAPL','AMZN','GOOG','MSFT','TSLA']
tickerStr = "AAPL"

model = load_model(os.path.join("..","static","models",f"{tickerStr}.h5"))

ticker = yf.Ticker(tickerStr)
df = ticker.history(period='5y')['Close'].values
df = df.reshape(-1, 1)
dataset_train = np.array(df[:int(df.shape[0]*0.8)])
dataset_test = np.array(df[int(df.shape[0]*0.8)-50:])
scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train)
dataset_test = scaler.transform(dataset_test)

def create_my_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i,0])
        y.append(df[i,0])
    x = np.array(x)
    y = np.array(y)
    return x,y

x_train, y_train = create_my_dataset(dataset_train)
x_test, y_test = create_my_dataset(dataset_test)

#reshaping for LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

predictions_prepended = [0 for i in range(0,df.shape[0]-len(predictions))]
predictions_prepended.extend(predictions.flatten().tolist())

overall = {
    "closing_prices": df.flatten().tolist(),
    "predicted_prices_total": predictions_prepended,
    "predicted_prices": predictions.flatten().tolist()
}
overall
#overall.to_json(os.path.join("..","static","data",f"{tickerStr}_overall.csv"),index_label="Index")

{'closing_prices': [26.585155487060547,
  25.80864143371582,
  26.231359481811523,
  26.11878776550293,
  26.94584083557129,
  27.288148880004883,
  27.407615661621094,
  27.051517486572266,
  27.31112289428711,
  27.1158447265625,
  27.065305709838867,
  27.177871704101562,
  26.957326889038086,
  26.713804244995117,
  26.4656925201416,
  27.3455810546875,
  27.17328453063965,
  27.161794662475586,
  26.562177658081055,
  26.688533782958984,
  26.001623153686523,
  25.840808868408203,
  25.38362693786621,
  25.57890510559082,
  25.03672981262207,
  24.359006881713867,
  24.65766143798828,
  24.634689331054688,
  24.951725006103516,
  24.8184757232666,
  24.540494918823242,
  24.98158836364746,
  24.65536117553711,
  24.182106018066406,
  24.202783584594727,
  23.59627914428711,
  23.13450813293457,
  22.15812110900879,
  22.27529525756836,
  22.63597869873047,
  22.964500427246094,
  22.374074935913086,
  22.863412857055664,
  22.314340591430664,
  22.206371307373047,
  22.23623657226